In [13]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from openai import OpenAI

def update_google_sheet(service, spreadsheet_id, sheet_name, titles, descriptions, scores):
    # Consolidate updates in a batch request
    batch_update_data = [
        {"range": f"{sheet_name}!B2:B", "values": [[title] for title in titles]},
        {"range": f"{sheet_name}!C2:C", "values": [[description] for description in descriptions]},
        {"range": f"{sheet_name}!H2:H", "values": [[score] for score in scores]}
    ]

    service.spreadsheets().values().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={"valueInputOption": "USER_ENTERED", "data": batch_update_data}
    ).execute()

def generate_captions_and_update_sheet(service, spreadsheet_id, sheet_name, client):
    # Fetch descriptions
    descriptions = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id, range=f"{sheet_name}!C2:C"
    ).execute().get('values', [])

    # Fetch the entire column E starting from E2
    entire_column_E = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id, range=f"{sheet_name}!E2:E"
    ).execute().get('values', [])

    # Concatenate all the text in column E
    concatenated_text = ' '.join([cell[0] for cell in entire_column_E if cell])

    print(f"Number of descriptions: {len(descriptions)}")

    # Generate captions
    captions = []
    for i, description in enumerate(descriptions):
        try:
            if description:
                prompt = f"Write a caption about the following around 100 words , starting with one emoticon: {description[0]}"
                response = client.chat.completions.create(
                    model="gpt-4", 
                    messages=[{"role": "system", "content": "You are a creative assistant."},
                              {"role": "user", "content": prompt}]
                )
                caption = response.choices[0].message.content.strip() + " " + concatenated_text
                captions.append([caption])
                print(f"Processed description {i+1}: {caption}") 
            else:
                print(f"Empty description at row {i+2}") 
        except Exception as e:
            print(f"Error processing description at row {i+2}: {e}")

    # Update the sheet with the new captions
    if captions:
        caption_range = f"{sheet_name}!D2:D{1+len(captions)}"
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id, range=caption_range,
            valueInputOption="USER_ENTERED", body={'values': captions}
        ).execute()
        
# Setup ChromeDriver
chrome_driver_path = 'C:\\Users\\djyan\\Downloads\\chromedriver-win32\\chromedriver-win32\\chromedriver.exe'
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Setup Google Sheets API
creds = Credentials.from_authorized_user_file('C:\\Users\\djyan\\Downloads\\client_secret_204709315294-t82p97ghvt54els62217rlgp3fj1d6jo.apps.googleusercontent.com (1).json', scopes=['https://www.googleapis.com/auth/spreadsheets'])
service = build('sheets', 'v4', credentials=creds)

# Spreadsheet ID and Sheet Name
spreadsheet_id = '1tdwufrfmL0Bc2lnehOEopSo9T9w717LvbsjNhT8xqrE'
sheet_name = "Copy of Tim Draper"

# Setup OpenAI API Client
openai_client = OpenAI()

try:
    driver.get("https://clip.opus.pro/clip/P0123005tWpp")
    wait = WebDriverWait(driver, 20)
    wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-base.css-1wc18uy")))

    titles = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-xl2.css-1jfjns0")
    descriptions = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-base.css-1wc18uy")
    scores = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-xl3.css-tz3tqc")

    titles_list, descriptions_list, scores_list = [], [], []
    for title, description, score in zip(titles, descriptions, scores):
        score_value = int(score.text)
        if score_value > 80:
            titles_list.append(title.text)
            descriptions_list.append(description.text)
            scores_list.append(str(score_value))

    # Sort by scores and update the sheet
    combined = sorted(zip(titles_list, descriptions_list, scores_list), key=lambda x: int(x[2]), reverse=True)
    sorted_titles, sorted_descriptions, sorted_scores = zip(*combined)

    update_google_sheet(service, spreadsheet_id, sheet_name, sorted_titles, sorted_descriptions, sorted_scores)
    generate_captions_and_update_sheet(service, spreadsheet_id, sheet_name, openai_client)

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    driver.quit()

Number of descriptions: 13
Processed description 1: 😲 This captivating video presents a transformative approach to traditional education, impeccably delivered by a speaker with both personal insight and deep connections. It underscores the need for innovation, expertly showcasing Draper University as a credible alternative. While it makes a compelling point about existing system flaws, a stronger call to action could amplify its impact. A must-watch for anyone interested in education's future landscape! #GenFIRE #SanEng #10Cs #GenerationalWealth #FinancialIndependence
Processed description 2: 🔥 This video is a game-changer for anyone struggling with the practical application of math! An engaging hook, clear structure and relatable content make math meaningful, bridging the gap left in traditional education. With a look at trends in improving education, this video is remarkably relevant. A call to action would have been the cherry on top, but it's an impressive piece overall that connec